In [1]:
import yfinance as yf
import pandas as pd
pd.set_option('display.max_rows', 200)

#### Warning: I edited yfinance's *base* as per this post to fix an issue 
https://github.com/ranaroussi/yfinance/pull/371/files/7e137357296a1df177399d26543e889848efc021

In [2]:
t = yf.Ticker("4001.SR")

In [3]:
dir(t)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_balancesheet',
 '_base_url',
 '_calendar',
 '_cashflow',
 '_download_options',
 '_earnings',
 '_expirations',
 '_financials',
 '_fundamentals',
 '_get_fundamentals',
 '_history',
 '_info',
 '_institutional_holders',
 '_isin',
 '_major_holders',
 '_options2df',
 '_recommendations',
 '_scrape_url',
 '_sustainability',
 'actions',
 'balance_sheet',
 'balancesheet',
 'calendar',
 'cashflow',
 'dividends',
 'earnings',
 'financials',
 'get_actions',
 'get_balance_sheet',
 'get_balancesheet',
 'get_calendar',
 'get_cashflow',
 'get_dividends',
 'get_earnings',
 'get_financials',
 'get_info',
 'get_institutional_holders',
 'ge

## Hist

In [4]:
t.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-03-04,8.20,8.20,8.09,8.09,300148.0,0.0,0.0
2010-03-07,8.13,8.20,8.09,8.16,293744.0,0.0,0.0
2010-03-08,8.24,8.24,8.16,8.16,561004.0,0.0,0.0
2010-03-09,8.20,8.20,8.09,8.16,408404.0,0.0,0.0
2010-03-10,8.16,8.20,8.16,8.16,230664.0,0.0,0.0
...,...,...,...,...,...,...,...
2020-08-06,119.20,119.80,119.00,119.20,56895.0,0.0,0.0
2020-08-09,119.00,120.00,118.60,119.00,102734.0,0.0,0.0
2020-08-10,118.20,120.00,118.00,120.00,84287.0,0.0,0.0


## Info

In [5]:
t.info

{'zip': '11531',
 'sector': 'Consumer Cyclical',
 'longBusinessSummary': 'Abdullah Al-Othaim Markets Company, together with its subsidiaries, engages in the wholesale and retail trade of food supplies and other products in the Kingdom of Saudi Arabia and Egypt. The company imports, exports, wholesales, and retails food products, wheat, rice, fish, meat, dairy products, ghee, olive, halawa, pasta, and soft drinks, as well as fruits, vegetables, and agricultural products; livestock and household items; and ready-made clothes, sport clothes, jewelry, sewing tools, bags, leather products, decorations, dropped ceilings, and vehicles spare parts. It also establishes, operates, manages, and maintains supermarkets, commercial complexes, bakeries, and cafes; and provides catering, as well as labor services regarding house workers and employment in the public and private sectors. In addition, the company maintains training and educational, entertainment, and sports centers; offers computer servi

In [2]:
import pymongo as pm
import json

In [3]:
client = pm.MongoClient("mongodb://localhost:27017/")
db = client["tadawul"]
companies = db["companies"]

In [4]:
# All relvant variables we're going to extract from each company
columns = ["city", "industry", "twoHundredDayAverage", "trailingAnnualDividendYield", "payoutRatio", "averageDailyVolume10Day", "fiftyDayAverage", "trailingAnnualDividendRate", "averageVolume10days", "dividendRate", "trailingPE", "averageVolume", "priceToSalesTrailing12Months", "fiftyTwoWeekHigh", "forwardPE", "fiveYearAvgDividendYield", "dividendYield", "symbol", "profitMargins", "52WeekChange", "forwardEps", "bookValue"]


In [12]:
# DataFrame of comapnies and their ID, Name and sector
# Source: https://www.mubasher.info/countries/sa/companies
# Date: 2020/8/12
with open("site.html", "r", encoding="utf-8") as f: 
    html = f.read()
companiesdf = pd.concat(pd.read_html(html, encoding="utf-8")) 
companiesdf = companiesdf.iloc[:, 0:3] # Remove non-relvant columns
companiesdf = companiesdf.reset_index(drop=True)

In [126]:
stockIds = db["stockIds"]
stockIds.insert_many(companiesdf.to_dict("records"))

In [6]:
for i, row in companiesdf.iterrows():
    try:
        document = {}
        ticker = yf.Ticker(str(row.loc["السهم"]) + ".SR")
            
        for col in columns:
                if ticker.info.get(col, None) is None: continue
                document.update({col: ticker.info[col]})

        history = ticker.history(period="max")
        history.index = history.index.astype(str)
        document.update({"history": history.to_dict()})

        companies.insert_one(document)
    except Exception as err:
        print(f"\nSkipped\t{str(row)}\nError:\t{err}\n")
        

- 2370.SR: 1d data not available for startTime=-2208988800 and endTime=1597536430. Only 100 years worth of day granularity data are allowed to be fetched per request.

Skipped	الشركة    زين السعودية
السهم             7030
القطاع       الإتصالات
Name: 110, dtype: object
Error:	'regularMarketOpen'


Skipped	الشركة      أسترا الصناعية
السهم                 1212
القطاع    السلع الرأسمالية
Name: 121, dtype: object
Error:	'regularMarketOpen'

- 8312.SR: No data found, symbol may be delisted
- 2222.SR: 1d data not available for startTime=-2208988800 and endTime=1597536607. Only 100 years worth of day granularity data are allowed to be fetched per request.

Skipped	الشركة                   عطاء
السهم                    4292
القطاع    الخدمات الإستهلاكية
Name: 160, dtype: object
Error:	'regularMarketOpen'


Skipped	الشركة             الواحة
السهم                3007
القطاع    المواد الأساسية
Name: 161, dtype: object
Error:	'regularMarketOpen'

- 4005.SR: 1d data not available for startTime=-220

In [7]:
companies.count_documents({})

202

In [7]:
#companies.delete_many({})

In [8]:
db.command("collstats", "events") 

{'ns': 'tadawul.events',
 'size': 0,
 'count': 0,
 'storageSize': 0,
 'totalSize': 0,
 'nindexes': 0,
 'totalIndexSize': 0,
 'indexDetails': {},
 'indexSizes': {},
 'scaleFactor': 1,
 'ok': 1.0}

In [9]:
db.command("dbstats") 

{'db': 'tadawul',
 'collections': 3,
 'views': 0,
 'objects': 391,
 'avgObjSize': 147645.81329923274,
 'dataSize': 57729513.0,
 'storageSize': 20721664.0,
 'indexes': 3,
 'indexSize': 69632.0,
 'totalSize': 20791296.0,
 'scaleFactor': 1.0,
 'fsUsedSize': 797010726912.0,
 'fsTotalSize': 890696101888.0,
 'ok': 1.0}

In [14]:
a = next(companies.find(sort=[("dividendYield", -1)]))

h = a.pop("history")
print(pd.DataFrame(h))
a

             Open   High    Low  Close     Volume  Dividends  Stock Splits
2012-01-17  13.30  13.40  13.07  13.13   846547.0        0.0           0.0
2012-01-18  13.20  13.66  13.10  13.53  1028786.0        0.0           0.0
2012-01-22  13.53  13.53  13.53  13.53        0.0        0.0           0.0
2012-01-23  13.53  13.53  13.53  13.53        0.0        0.0           0.0
2012-01-24  13.56  13.76  13.50  13.63  1059553.0        0.0           0.0
...           ...    ...    ...    ...        ...        ...           ...
2020-08-09  21.70  22.22  21.58  22.14   718636.0        0.0           0.0
2020-08-10  22.14  22.40  21.96  22.10   426081.0        0.0           0.0
2020-08-11  22.12  23.60  22.00  22.82  1959304.0        0.0           0.0
2020-08-12  22.86  23.12  22.80  22.98   254740.0        0.0           0.0
2020-08-13  22.98  22.98  22.98  22.98        0.0        0.0           0.0

[2113 rows x 7 columns]


{'_id': ObjectId('5f38793503d4ca362c0da189'),
 'city': 'Riyadh',
 'industry': 'Building Products & Equipment',
 'twoHundredDayAverage': 18.86961,
 'trailingAnnualDividendYield': 0.021758052,
 'payoutRatio': 0.33330002,
 'averageDailyVolume10Day': 696301,
 'fiftyDayAverage': 22.098715,
 'trailingAnnualDividendRate': 0.5,
 'averageVolume10days': 696301,
 'dividendRate': 10,
 'trailingPE': 30.171278,
 'averageVolume': 433791,
 'priceToSalesTrailing12Months': 1.0022023,
 'fiftyTwoWeekHigh': 26.7,
 'forwardPE': 32.25352,
 'dividendYield': 0.4352,
 'symbol': '1301.SR',
 'profitMargins': 0.04151,
 '52WeekChange': 0.56753063,
 'forwardEps': 0.71,
 'bookValue': 12.447}

In [21]:
import numpy as np
np.array([[1, 2, 7], [3, 4, 8]]) @ np.array([[1, 2], [3, 9], [4, 16]])

array([[ 35, 132],
       [ 47, 170]])

In [129]:
pd.DataFrame(tuple(stockIds.find({})))

,_id,الشركة,السهم,القطاع
0,5f34e41b91c30b141f7377a7,سامبا,1090,البنوك
1,5f34e41b91c30b141f7377a8,السعودي الفرنسي,1050,البنوك
2,5f34e41b91c30b141f7377a9,الجزيرة,1020,البنوك
3,5f34e41b91c30b141f7377aa,الرياض,1010,البنوك
4,5f34e41b91c30b141f7377ab,ساب,1060,البنوك
5,5f34e41b91c30b141f7377ac,العربي,1080,البنوك
6,5f34e41b91c30b141f7377ad,الراجحي,1120,البنوك
7,5f34e41b91c30b141f7377ae,سيسكو,2190,النقل
8,5f34e41b91c30b141f7377af,المصافي,2030,الطاقة
9,5f34e41b91c30b141f7377b0,زجاج,2150,المواد الأساسية
